In [1]:
%load_ext autoreload
%autoreload 2
from gratin.models.main_net import MainNet
from gratin.data.datamodule import DataModule
import pytorch_lightning as pl
from gratin.models.utils import get_predictions_of_dl
from gratin.training.callbacks import Plotter
import matplotlib.pyplot as plt
from pytorch_lightning.callbacks import (
    EarlyStopping,
    LearningRateMonitor,
    ModelCheckpoint,
)
from gratin.data.dataset import ExpTrajDataSet
from torch_geometric.loader import DataLoader
import os
from umap import ParametricUMAP
import tensorflow as tf
import numpy as np
import logging
import torch.cuda
import pandas as pd
from typing import Union, List
import warnings
from gratin.models.MMD_net import MMDNet

logging.getLogger("pytorch_lightning").setLevel(logging.ERROR)

graph_info = {
    "edges_per_point": 10,
    "clip_trajs": False,
    "scale_types": ["step_std","mean_time_step"],
    "log_features": True,
    "data_type": "no_features",  # no features because features are all computed by the model
    "edge_method": "geom_causal",
}

In [5]:
model = MMDNet(n_c = 12,latent_dim=2,dim=2)

In [6]:
tb_logger = pl.loggers.TensorBoardLogger(
    save_dir=os.path.join("/home/hverdier/Gaia/hecat/hippo/gratin_mmd", "tb_logs"),
    default_hp_metric=False,
    name="model",
    flush_secs=60,
)

In [7]:
trainer = pl.Trainer(
        auto_select_gpus=torch.cuda.is_available(),
        gpus=1 * torch.cuda.is_available(),
        gradient_clip_val=10.0,
        reload_dataloaders_every_n_epochs=1,
        log_every_n_steps=50,
        max_epochs=1,
        detect_anomaly=True,
        track_grad_norm=2,
        logger=tb_logger,
    )

In [10]:
num_workers = 8
dim = 2
time_delta_range = (0.005,1.)
length_range = (7,30)
log_diffusion_range = (-2.,.5)
noise_range = (0.015,0.05)

In [11]:
dl_params = {"batch_size": 128, "num_workers": num_workers}

ds_params = {
    "dim": dim,  # can be (1, 2 or 3)
    "RW_types": [
        "fBM",
        "LW",
        "sBM",
        "OU",
        "CTRW",
    ],  # Types of random walks used during training
    "time_delta_range": time_delta_range,
    "logdiffusion_range": log_diffusion_range,
    "length_range": length_range,
    "noise_range": noise_range,
    "N": int(1e5),
}

In [13]:
pl.seed_everything(1)
dm = DataModule(ds_params=ds_params, dl_params=dl_params, graph_info=graph_info)
dm.setup(plot=False)


stage is None, strange...


In [15]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    trainer.fit(model, dm)

Training: 37it [00:00, ?it/s]

ValueError: 
 you tried to log 0.019700000062584877 which is currently not supported. Try a dict or a scalar/tensor.

In [ ]:
model.eval()
h = get_predictions_of_dl(
    model, dm.test_dataloader(), latent_samples=int(3e4)
)